In [2]:
import pandas as pd

In [6]:
submissions = pd.read_csv("submissions.csv")
users = pd.read_csv("users.csv")
assignments = pd.read_csv("assignments.csv")
courses = pd.read_csv("courses.csv")
deadlines = pd.read_csv("deadlines.csv")
enrollments = pd.read_csv("enrollments.csv")

# Merge steps
df = submissions.merge(users, on='user_id', suffixes=('', '_student'))
df = df.merge(assignments, on='assignment_id')
df = df.merge(deadlines, on='assignment_id', how='left')
df = df.merge(courses, on='course_id')

# Join with professor info (self join on users table)
df = df.merge(users, left_on='professor_id', right_on='user_id', suffixes=('', '_professor'))



# (Optional) Join enrollments if needed
# df = df.merge(enrollments, left_on=['course_id', 'user_id'], right_on=['course_id', 'student_id'], how='left')

# Save to a single CSV
df.to_csv("full_combined_data.csv", index=False)

     submission_id  user_id  assignment_id submission_date  grade  \
0                1      112              1      2025-07-26     81   
1                2        4              1      2025-08-03     37   
2                3       63              1      2025-07-26     63   
3                4       31              1      2025-07-27     96   
4                5       72              1      2025-08-01     67   
..             ...      ...            ...             ...    ...   
391            392       44             30      2025-07-21     45   
392            393       40             30      2025-07-30     59   
393            394       52             30      2025-07-23     55   
394            395       18             30      2025-08-01      0   
395            396       38             30      2025-07-25     56   

                 name                        email     role  course_id  \
0       Michael Davis        richard54@example.net  student         13   
1      Brandon Kelley  

In [8]:
import pandas as pd

# --- 1. Read Data from CSV Files ---

# Users Table
# Contains information about all users (students and professors)
users_df = pd.read_csv('users.csv')
print("--- Users Table (from CSV) ---")
print(users_df)
print("\n")

# Courses Table
# Contains information about courses, including the professor teaching it
courses_df = pd.read_csv('courses.csv')
print("--- Courses Table (from CSV) ---")
print(courses_df)
print("\n")

# Enrollments Table
# Links students to courses
enrollments_df = pd.read_csv('enrollments.csv')
print("--- Enrollments Table (from CSV) ---")
print(enrollments_df)
print("\n")

# Assignments Table
# Assignments for each course
assignments_df = pd.read_csv('assignments.csv')
print("--- Assignments Table (from CSV) ---")
print(assignments_df)
print("\n")

# Deadlines Table
# Specific deadlines and reminder dates for assignments (can override assignment due_date)
deadlines_df = pd.read_csv('deadlines.csv')
print("--- Deadlines Table (from CSV) ---")
print(deadlines_df)
print("\n")

# Submissions Table
# Records of student submissions for assignments
submissions_df = pd.read_csv('submissions.csv')
print("--- Submissions Table (from CSV) ---")
print(submissions_df)
print("\n")

# --- 2. Merge Tables into a Single Denormalized Table ---

# Start with enrollments as the base, as it links students to courses.
# Each row in the final table will represent an enrollment, potentially expanded
# by assignments and submissions related to that enrollment.

# Merge enrollments with student user details
# Rename 'name', 'email', 'role' from users_df to avoid conflict with professor details later
denormalized_df = pd.merge(
    enrollments_df,
    users_df.rename(columns={'name': 'student_name', 'email': 'student_email', 'role': 'student_role'}),
    left_on='student_id',
    right_on='user_id',
    how='left'
)
# Drop the redundant 'user_id' column from the merge
denormalized_df.drop(columns=['user_id'], inplace=True)

# Merge with course details
denormalized_df = pd.merge(
    denormalized_df,
    courses_df.rename(columns={'start_date': 'course_start_date'}),
    on='course_id',
    how='left'
)

# Merge with professor details (joining users table again, but for professors)
denormalized_df = pd.merge(
    denormalized_df,
    users_df.rename(columns={'user_id': 'professor_id', 'name': 'professor_name', 'email': 'professor_email', 'role': 'professor_role'}),
    on='professor_id',
    how='left',
    suffixes=('_student', '_professor') # Suffixes to distinguish if original column names were same
)

# Merge with assignments for the course.
# This will create multiple rows for each enrollment if there are multiple assignments.
denormalized_df = pd.merge(
    denormalized_df,
    assignments_df.rename(columns={'due_date': 'assignment_original_due_date'}),
    on='course_id',
    how='left'
)

# Merge with deadlines (optional, as not all assignments might have a separate deadline entry)
denormalized_df = pd.merge(
    denormalized_df,
    deadlines_df.rename(columns={'due_date': 'deadline_actual_due_date'}),
    on='assignment_id',
    how='left'
)

# Merge with submissions.
# This will further expand rows if a student has multiple submissions for an assignment
# (though typically there's one final submission).
# We need to join on both student_id (which is user_id in submissions) and assignment_id.
denormalized_df = pd.merge(
    denormalized_df,
    submissions_df.rename(columns={'user_id': 'student_id_for_submission'}), # Rename to avoid conflict
    left_on=['student_id', 'assignment_id'],
    right_on=['student_id_for_submission', 'assignment_id'],
    how='left',
    suffixes=('_assignment', '_submission')
)
# Drop the redundant 'student_id_for_submission' column
denormalized_df.drop(columns=['student_id_for_submission'], inplace=True)


# --- 3. Final Cleaning and Column Reordering (Optional but Recommended) ---

# Select and reorder columns for better readability
final_columns = [
    'enrollment_id',
    'student_id',
    'student_name',
    'student_email',
    'student_role',
    'course_id',
    'course_name',
    'course_start_date',
    'professor_id',
    'professor_name',
    'professor_email',
    'professor_role',
    'assignment_id',
    'assignment_title',
    'assignment_original_due_date',
    'deadline_actual_due_date',
    'deadline_reminder_date',
    'submission_id',
    'submission_date',
    'grade' # 'grade' came from submissions_df
]

# Ensure all columns exist before selecting, fill missing with None or NaN
for col in final_columns:
    if col not in denormalized_df.columns:
        denormalized_df[col] = None # Or pd.NA for better type handling

denormalized_df = denormalized_df[final_columns]

# Convert relevant columns to appropriate data types if necessary
# For example, dates can be converted to datetime objects
# denormalized_df['course_start_date'] = pd.to_datetime(denormalized_df['course_start_date'])
# denormalized_df['assignment_original_due_date'] = pd.to_datetime(denormalized_df['assignment_original_due_date'])
# denormalized_df['deadline_actual_due_date'] = pd.to_datetime(denormalized_df['deadline_actual_due_date'])
# denormalized_df['deadline_reminder_date'] = pd.to_datetime(denormalized_df['deadline_reminder_date'])
# denormalized_df['submission_date'] = pd.to_datetime(denormalized_df['submission_date'])

print("--- Denormalized Single Table ---")
print(denormalized_df.to_string()) # Use to_string() to display all rows/columns without truncation

# You can save this DataFrame to a CSV, Excel, or other formats:
# denormalized_df.to_csv('denormalized_academic_data.csv', index=False)
# denormalized_df.to_excel('denormalized_academic_data.xlsx', index=False)
denormalized_df.to_csv("denormalised.csv",index=False)


--- Users Table (from CSV) ---
     user_id                name                      email       role
0          1      William Bailey        fgarcia@example.com    student
1          2        Rachel Cross  tamaramcclure@example.net    student
2          3  Christopher Harris     hhenderson@example.org    student
3          4      Brandon Kelley     ihernandez@example.org    student
4          5       Lauren Pierce         mark07@example.com    student
..       ...                 ...                        ...        ...
127      128     Taylor Humphrey        aharvey@example.com  professor
128      129   Michael Rodriguez    donnamiller@example.net  professor
129      130   Elizabeth Fleming       jeremy64@example.net  professor
130      131       Taylor Travis     davisjulia@example.com  professor
131      132          Angela Cox   thomasarnold@example.org  professor

[132 rows x 4 columns]


--- Courses Table (from CSV) ---
    course_id               course_name  professor_id  sta